# Opciones Americanas

In [1]:
import QuantLib as ql



Trabajemos ahora con Opciones Americanas. Las opciones americanas recoremos que tienene ejercicio temprano. Es decir uno puede ejercerlas antes de vencimiento. Esa opcionalidad añade un ingrediente de complejidad, dado que hay que calcular el ejercicio optimo para poder hacer una buena valuacion.
Aqui el modelo de Black Scholes no lo podremos usar.
Son pocas las cosas que hay que modificar. Primero redefinimos el tipo de ejercicio a un ejercicio de tipo americano (`AmericanExercise`)


In [2]:
#Definimos los inputs
precio_activo = 440 #Spot (S)
precio_ejercicio = 450 #Strike (K)
volatilidad = 0.3394  #Volatilidad Implicita anualizada (sigma)
tasa_interes = 0.0013 #tasa libre de riesgo anualizada (r)
tasa_dividendos =  0.075 #tasa de dividendos anualizada (q)

#Expiracion el 16 de octubre de 2020
fecha_expiracion = ql.Date(16, 10, 2020)
#Valuacion el 16 de octubre de 2020
fecha_valuacion = ql.Date(6, 8, 2020)
ql.Settings.instance().evaluationDate = fecha_valuacion
day_count = ql.Actual365Fixed()
calendario = ql.UnitedStates()


En este punto es en el que se invocan nuevas clases. En particular: `AmericanExercise` que a diferencia de `EuropeanExercise` pide tambien la fecha de valuacion como entrada

In [3]:
tipo_opcion = ql.Option.Call #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)
ejercicio_americano = ql.AmericanExercise(fecha_valuacion, fecha_expiracion)
opcion_americana = ql.VanillaOption(payoff, ejercicio_americano)

Cramos los objetos necesarios para la valuacion (notar que son identicos que en el caso Europeo.)

In [4]:
S = ql.SimpleQuote(precio_activo)
objeto_spot = ql.QuoteHandle(S)

r = ql.SimpleQuote(tasa_interes)
objeto_tasa_interes = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                            ql.QuoteHandle(r), 
                                                        day_count))
q = ql.SimpleQuote(tasa_dividendos)
objeto_tasa_dividendos = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                      ql.QuoteHandle(q), 
                                                      day_count))

sigma = ql.SimpleQuote(volatilidad)
objeto_volatilidad = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(fecha_valuacion, 
                                                                 calendario, 
                                                                 ql.QuoteHandle(sigma), 
                                                             day_count))

proceso_BSM = ql.BlackScholesMertonProcess(objeto_spot, 
                                           objeto_tasa_dividendos, 
                                           objeto_tasa_interes, 
                                           objeto_volatilidad)

El siguiente paso es montar el proceso a un modelo (engine) de precios. Veamos que si intentamos usar por ejemplo Black Scholes QuantLib tira un logico error [not an European option], dado que BS es solamente para opciones Europeas:


In [6]:
opcion_americana.setPricingEngine(ql.AnalyticEuropeanEngine(proceso_BSM))
precio_opcion_BS_Ame = opcion_americana.NPV()

RuntimeError: not an European option

Para un pricer 'estilo' BS podemos invocar al modelo de Trabajamos entonces con dos alternativa: El modelo de aproximacion de Bjerksund y Stensland `BjerksundStenslandEngine` y el de Barone Adesi y Whaley `BaroneAdesiWhaleyEngine`

In [8]:
opcion_americana.setPricingEngine(ql.BaroneAdesiWhaleyEngine(proceso_BSM))
precio_opcion_BAW_Ame = opcion_americana.NPV()

opcion_americana.setPricingEngine(ql.BjerksundStenslandEngine(proceso_BSM))
precio_opcion_BjSt_Ame = opcion_americana.NPV()

print(precio_opcion_BAW_Ame, precio_opcion_BjSt_Ame)


19.4053805016032 19.217979133477456


C:\Users\manue\anaconda3\lib\site-packages\QuantLib\QuantLib.py:12696: UserWarning: BaroneAdesiWhaleyEngine is deprecated; use BaroneAdesiWhaleyApproximationEngine
  warn('%s is deprecated; use %s' % (BaroneAdesiWhaleyEngine.__name__, BaroneAdesiWhaleyApproximationEngine.__name__))
C:\Users\manue\anaconda3\lib\site-packages\QuantLib\QuantLib.py:12713: UserWarning: BjerksundStenslandEngine is deprecated; use BjerksundStenslandApproximationEngine
  warn('%s is deprecated; use %s' % (BjerksundStenslandEngine.__name__, BjerksundStenslandApproximationEngine.__name__))


Invoco ahora a los otros tipos de pricer vistos en el caso Europeo. Desde ya estos son modificaciones de los anteriores.

In [9]:
# No hay Black Scholes

modelo_arbol = 'CRR'
pasos_arbol = 1000

opcion_americana.setPricingEngine(ql.BinomialVanillaEngine(proceso_BSM, 
                                                           modelo_arbol, 
                                                           pasos_arbol))
precio_opcion_Bin_Ame = opcion_americana.NPV()

In [10]:
opcion_americana.setPricingEngine(ql.FdBlackScholesVanillaEngine(proceso_BSM))
precio_opcion_DF_Ame = opcion_americana.NPV()

pasos_tiempo = 20
caminos = 50000

opcion_americana.setPricingEngine(ql.MCAmericanEngine(proceso_BSM, 
                                                      "PseudoRandom", 
                                                      timeSteps = pasos_tiempo, 
                                                      requiredSamples = caminos))
precio_opcion_MC_Ame = opcion_americana.NPV()

In [11]:
print("Precio modelo Amercana Bjerksund Stensland QuantLib:", precio_opcion_BjSt_Ame)
print("Precio modelo Amercana Barone Adesi Whaley QuantLib:", precio_opcion_BAW_Ame)
print("Precio modelo Amercana Binomial QuantLib:", precio_opcion_Bin_Ame)
print("Precio modelo Amercana Montecarlo QuantLib:", precio_opcion_MC_Ame)
print("Precio modelo Amercana Dif. FinitasQuantLib:", precio_opcion_DF_Ame)

Precio modelo Amercana Bjerksund Stensland QuantLib: 19.217979133477456
Precio modelo Amercana Barone Adesi Whaley QuantLib: 19.4053805016032
Precio modelo Amercana Binomial QuantLib: 19.430458475988022
Precio modelo Amercana Montecarlo QuantLib: 19.402791275559714
Precio modelo Amercana Dif. FinitasQuantLib: 19.41200863593298


## Diferencias de precios entre opciones Europeas y Americanas

### Caso Call sin dividendos

En este caso no hay diferencia de precios

In [12]:

#Definimos los inputs
precio_activo = 100 #Spot (S)
precio_ejercicio = 120 #Strike (K)
volatilidad = 0.3  #Volatilidad Implicita anualizada (sigma)
tasa_interes = 0.001 #tasa libre de riesgo anualizada (r)
tasa_dividendos =  0.0 #tasa de dividendos anualizada (q)


#Expiracion el 16 de octubre de 2020
fecha_expiracion = ql.Date(16, 10, 2020)
#Valuacion el 16 de octubre de 2020
fecha_valuacion = ql.Date(6, 8, 2020)
ql.Settings.instance().evaluationDate = fecha_valuacion
day_count = ql.Actual365Fixed()
calendario = ql.UnitedStates()


S = ql.SimpleQuote(precio_activo)
objeto_spot = ql.QuoteHandle(S)

r = ql.SimpleQuote(tasa_interes)
objeto_tasa_interes = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                            ql.QuoteHandle(r), 
                                                        day_count))

q = ql.SimpleQuote(tasa_dividendos)
objeto_tasa_dividendos = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                      ql.QuoteHandle(q), 
                                                      day_count))

sigma = ql.SimpleQuote(volatilidad)
objeto_volatilidad = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(fecha_valuacion, 
                                                                 calendario, 
                                                                 ql.QuoteHandle(sigma), 
                                                             day_count))

proceso_BSM = ql.BlackScholesMertonProcess(objeto_spot, 
                                           objeto_tasa_dividendos, 
                                           objeto_tasa_interes, 
                                           objeto_volatilidad)



tipo_opcion = ql.Option.Call #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)


ejercicio_americano = ql.AmericanExercise(fecha_valuacion, fecha_expiracion)
opcion_americana = ql.VanillaOption(payoff, ejercicio_americano)

ejercicio_europeo = ql.EuropeanExercise(fecha_expiracion)
opcion_europea = ql.VanillaOption(payoff, ejercicio_europeo)


modelo_arbol = 'CRR'
pasos_arbol = 1000

modelo = ql.BinomialVanillaEngine(proceso_BSM, modelo_arbol, pasos_arbol)


opcion_americana.setPricingEngine(modelo)
opcion_europea.setPricingEngine(modelo)





In [13]:
precio_americana = opcion_americana.NPV()
precio_europea = opcion_europea.NPV()

print("Precio modelo Amercana:", precio_americana)
print("Precio modelo Europea:", precio_europea)

Precio modelo Amercana: 0.5585418910215348
Precio modelo Europea: 0.5585418910215348


### Caso Call con dividendos



In [14]:
tasa_dividendos =  0.05 #tasa de dividendos anualizada (q)
q.setValue(tasa_dividendos)

precio_americana = opcion_americana.NPV()
precio_europea = opcion_europea.NPV()

print("Precio modelo Americana:", precio_americana)
print("Precio modelo Europea:", precio_europea)

Precio modelo Americana: 0.4765140777277043
Precio modelo Europea: 0.4724121922029323


### Caso Put con dividendos

In [15]:
tipo_opcion = ql.Option.Put #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)

ejercicio_americano = ql.AmericanExercise(fecha_valuacion, fecha_expiracion)
opcion_americana = ql.VanillaOption(payoff, ejercicio_americano)

ejercicio_europeo = ql.EuropeanExercise(fecha_expiracion)
opcion_europea = ql.VanillaOption(payoff, ejercicio_europeo)


tasa_dividendos =  0.05 #tasa de dividendos anualizada (q)
q.setValue(tasa_dividendos)

opcion_americana.setPricingEngine(modelo)
opcion_europea.setPricingEngine(modelo)

In [16]:
precio_americana = opcion_americana.NPV()
precio_europea = opcion_europea.NPV()

print("Precio modelo Americana:", precio_americana)
print("Precio modelo Europea:", precio_europea)

Precio modelo Americana: 21.416968770127703
Precio modelo Europea: 21.416968770127703


### Caso Put sin dividendos

In [17]:

tasa_dividendos =  0.00 #tasa de dividendos anualizada (q)
q.setValue(tasa_dividendos)


#opcion_americana.setPricingEngine(modelo)
#opcion_europea.setPricingEngine(modelo)


precio_americana = opcion_americana.NPV()
precio_europea = opcion_europea.NPV()

print("Precio modelo Americana:", precio_americana)
print("Precio modelo Europea:", precio_europea)

Precio modelo Americana: 20.54036429016482
Precio modelo Europea: 20.535202917603172
